In [0]:
%sql
SELECT
pickup_zip,
COUNT(*) AS total_trips,
AVG(fare_amount) AS avg_fare_amount,
AVG(trip_distance) AS avg_trip_distance
FROM
mc_labs.trips_raw
GROUP BY
pickup_zip
ORDER BY
total_trips DESC; -- Ordena pelos CEPs com mais viagens

In [0]:
%sql
WITH
ranked_trips AS (
  SELECT
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    trip_distance,
    fare_amount,
    pickup_zip,
    dropoff_zip,
    ROW_NUMBER() OVER (
      PARTITION BY
        tpep_pickup_datetime,
        tpep_dropoff_datetime,
        trip_distance,
        fare_amount -- Usando uma combinação de campos para identificar "duplicatas"
      ORDER BY
        tpep_pickup_datetime -- Ordem para decidir qual "duplicata" manter
    ) AS rn
  FROM
    mc_labs.trips_raw
)
SELECT
tpep_pickup_datetime,
tpep_dropoff_datetime,
trip_distance,
fare_amount,
pickup_zip,
dropoff_zip -- Lista explicitamente todas as colunas que você quer manter, exceto 'rn'
FROM
ranked_trips
WHERE
rn = 1;

In [0]:
%sql
SELECT
  CAST(tpep_pickup_datetime AS DATE) AS trip_date,
  COUNT(*) AS daily_total_trips,
  AVG(fare_amount) AS daily_avg_fare_amount,
  AVG(trip_distance) AS daily_avg_trip_distance
FROM
  mc_labs.trips_raw
GROUP BY
  CAST(tpep_pickup_datetime AS DATE)
ORDER BY
  trip_date;